# Grid Search hyper parameters for improved SRST

1. Import dependencies

In [1]:
import random
import torch
import numpy as np
import pandas as pd
import time
import glob
import os

from src.util.torch_device import resolve_torch_device
from src.data.indian_pines import load_indian_pines
from src.util.semi_guided import sample_fraction_from_segmentation
from src.definitions import GREED_SEARCH_FOLDER
from src.model.grid_search import GridSearch
from src.model.improved_spatial_regulated_self_training_grid_search import (
    ImprovedSpatialRegulatedSelfTrainingPipelineGridSearchAdapter,
)

2. Prepare env

In [2]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

generator = torch.Generator()
generator.manual_seed(random_seed)

In [3]:
f"Device is {device}"

'Device is mps'

# Indian pines (Cluster exponential decay)

0. Set params

In [4]:
examples_per_class = 0.7
epoch_seconds = int(time.time())
run_name = f"improved-indian-pines-cluster-exponential-decay-{epoch_seconds}"

In [5]:
cpu_count = 4

f"Setting num_workers to {cpu_count}"

'Setting num_workers to 4'

1. Load dataset

In [6]:
image, labels = load_indian_pines()

In [7]:
num_classes = len(np.unique(labels))

f"Number of classes {num_classes}"

'Number of classes 17'

In [8]:
masked_labels = sample_fraction_from_segmentation(labels, examples_per_class)

2. Train model

In [9]:
params = {
    "input_channels": [50, 100, 150, 200],
    "learning_rate": [1e-3, 1e-4],
    "patch_size": [9],
    "num_epochs": [11],
    "feature_extractor_epochs": [1, 9, 11],
    "semantic_threshold": [0.5, 0.6, 0.8],
    "lambda_v": [0.07, 0.09, 0.2, 0.4, 0.49],
    "k_star": [num_classes * 2, num_classes * 3],
    "batch_size": [64],
}

adapter = ImprovedSpatialRegulatedSelfTrainingPipelineGridSearchAdapter(
    params,
    image,
    masked_labels,
    labels,
    num_classes,
    device,
    random_seed,
    generator,
)

In [10]:
log_dir = GREED_SEARCH_FOLDER / run_name

In [11]:
search = GridSearch(
    adapter=adapter,
    optimize_metric="kappa_score",
    log_dir=log_dir,
    num_workers=cpu_count,
)

In [ ]:
_, best_params, best_score = search.run()

  2%|▏         | 4/180 [1:01:17<59:12:26, 1211.06s/it, best_score=0, split=1]0]

In [ ]:
print("Best Params:", best_params)
print("Best Score:", best_score)

3. Training results

In [ ]:
csv_files = glob.glob(os.path.join(log_dir, "*.csv"))

report = pd.concat([pd.read_csv(f) for f in csv_files])

report.head()

In [ ]:
len(report)

In [ ]:
report.sort_values("kappa_score", ascending=False).head()